In [ ]:
import os
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from itertools import combinations
import warnings
from google.colab import drive
import tensorflow as tf
from tensorflow import keras
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
from itertools import combinations
from tqdm import tqdm
import warnings
warnings.simplefilter('ignore')
from google.colab import drive
from scipy.sparse import coo_matrix
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import WikiCS
from torch_geometric.nn import aggr
import torch.nn.init as init
import torch.optim as optim

2.3.0+cu121
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import torch
def foo(a,ind,sz):
    idxs=torch.arange(sz)+ind
    return torch.index_select(a,0,idxs)

a=torch.arange(20).reshape((2,10))
c=torch.tensor([2,5])
print(torch.vmap(foo,in_dims=(0,0,None))(a,c,2)) # this is inside a vmap
print([foo(suba,subc,2) for (suba,subc) in zip(a,c)]) # but this is the same content

tensor([[ 2,  3],
        [15, 16]])
[tensor([2, 3]), tensor([15, 16])]


In [ ]:
dataset = WikiCS(root='path/to/dataset')

# Access the data
data = dataset[0]  # Access the first graph in the dataset
x = data.x  # Node features
edge_index = data.edge_index  # Edges
y = data.y  # Node labels
train_mask = data.train_mask  # Training mask
val_mask = data.val_mask  # Validation mask
test_mask = data.test_mask  # Test mask

In [ ]:
def get_train_split_from_bool(data) :
  data = data
  adjacency_list = [(i.item(), j.item()) for i, j in zip(data.edge_index[0], data.edge_index[1])]
  train_indices = data.train_mask[:, 0]  # assuming we're using the first training split
  train_indices = torch.where(train_indices)[0]

  X_train = data.x[train_indices]
  y_train = data.y[train_indices]

  # Create adjacency list for training set
  train_nodes = set(train_indices.tolist())
  adjacency_list_train = [(i, j) for i, j in adjacency_list if i in train_nodes and j in train_nodes]
  edge_index_train = torch.tensor(adjacency_list_train, dtype=torch.long).t().contiguous()

  return adjacency_list_train, edge_index_train,X_train,y_train

adjacency_list_train, edge_index_train,X_train,y_train = get_train_split_from_bool(data)

In [ ]:
def tensor_to_one_hot(labels, num_classes=None):
    # Determine the number of classes if not provided
    if num_classes is None:
        num_classes = labels.max().item() + 1
    # Convert labels to one-hot
    one_hot_labels = F.one_hot(labels, num_classes=num_classes)

    return one_hot_labels.float()

num_classes = 10
one_hot_labels = tensor_to_one_hot(y, num_classes)
one_hot_labels_train = tensor_to_one_hot(y_train, num_classes)

In [ ]:
model = nn.Sequential(
    nn.Linear(300,10),
    nn.Tanh()
)

optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()
# criterion = nn.MSELoss()
train_negtest = torch.logical_not(data.test_mask)
num_epoch = 500

for epoch in range(num_epoch):
  out = model(data.x)
  # loss = criterion(out, one_hot_labels)
  #train loss
  loss = criterion(out, one_hot_labels)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  if epoch % 10 == 0:
    print(f'Epoch {epoch}, Loss: {loss:.4f}')

Epoch 0, Loss: 2.3095
Epoch 10, Loss: 1.8067
Epoch 20, Loss: 1.5905
Epoch 30, Loss: 1.4849
Epoch 40, Loss: 1.4209
Epoch 50, Loss: 1.3820
Epoch 60, Loss: 1.3562
Epoch 70, Loss: 1.3375
Epoch 80, Loss: 1.3236
Epoch 90, Loss: 1.3130
Epoch 100, Loss: 1.3045
Epoch 110, Loss: 1.2977
Epoch 120, Loss: 1.2919
Epoch 130, Loss: 1.2871
Epoch 140, Loss: 1.2830
Epoch 150, Loss: 1.2795
Epoch 160, Loss: 1.2764
Epoch 170, Loss: 1.2737
Epoch 180, Loss: 1.2714
Epoch 190, Loss: 1.2694
Epoch 200, Loss: 1.2676
Epoch 210, Loss: 1.2661
Epoch 220, Loss: 1.2647
Epoch 230, Loss: 1.2634
Epoch 240, Loss: 1.2624
Epoch 250, Loss: 1.2614
Epoch 260, Loss: 1.2605
Epoch 270, Loss: 1.2597
Epoch 280, Loss: 1.2590
Epoch 290, Loss: 1.2584
Epoch 300, Loss: 1.2578
Epoch 310, Loss: 1.2573
Epoch 320, Loss: 1.2568
Epoch 330, Loss: 1.2564
Epoch 340, Loss: 1.2560
Epoch 350, Loss: 1.2557
Epoch 360, Loss: 1.2553
Epoch 370, Loss: 1.2550
Epoch 380, Loss: 1.2548
Epoch 390, Loss: 1.2545
Epoch 400, Loss: 1.2543
Epoch 410, Loss: 1.2541
Epo

In [ ]:
"""

0) Construct concatenate features matrix ---> concatenated_features_matrix
1) Calculate MLP scores of neighbors
2) Calculate the l1-distance of the pari-wise neighbors
3) Do Top-P Sampling using distances
3-1) Sort l1-distance while keeping the track of indices ---> concatenated_features_matrix : center_indices, neighbor_indices
4) adj matrix modified in a such way that only top-neighbors remained
5) find the not top-p neighbors and make the entry in adj matrix zero


6) Algorithm :
6-1) iterate over unique center nodes
6-2) create a mask that identify which indices are for each unique node
6-3) applying mask to sorted neighbor indices and l1-distances
6-4) choose the top-p neighbors according to the p provided for each node
6-5) use these selected neighbors to modify the adj matrix to refelect the sampled the neighbors


Top-P sampeling onlu differs between node relationship types.

Recom : partition 5 random groups
"""

'\n\n0) Construct concatenate features matrix ---> concatenated_features_matrix\n1) Calculate MLP scores of neighbors\n2) Calculate the l1-distance of the pari-wise neighbors\n3) Do Top-P Sampling using distances\n3-1) Sort l1-distance while keeping the track of indices ---> concatenated_features_matrix : center_indices, neighbor_indices\n4) adj matrix modified in a such way that only top-neighbors remained\n5) find the not top-p neighbors and make the entry in adj matrix zero\n\n\n6) Algorithm :\n6-1) iterate over unique center nodes\n6-2) create a mask that identify which indices are for each unique node\n6-3) applying mask to sorted neighbor indices and l1-distances\n6-4) choose the top-p neighbors according to the p provided for each node\n6-5) use these selected neighbors to modify the adj matrix to refelect the sampled the neighbors\n\n\nTop-P sampeling onlu differs between node relationship types.\n\nRecom : partition 5 random groups\n'

In [ ]:
def concatenate_features_optimized(feature_matrix, adjacency_matrix):
    num_nodes, num_features = feature_matrix.shape

    # Create indices for center nodes and their neighbors based on the adjacency matrix
    center_indices, neighbor_indices = torch.nonzero(adjacency_matrix, as_tuple=True)
    # print(center_indices, '\n',
    #       neighbor_indices)

    # Extract features for center nodes and their neighbors
    center_features = feature_matrix[center_indices]
    neighbor_features = feature_matrix[neighbor_indices]


    # Concatenate the features along dimension 1
    concatenated_features = torch.cat((center_features.unsqueeze(1), neighbor_features.unsqueeze(1)), dim=1)
    # print(concatenated_features)

    # Reshape to the desired output format (num_pairs, num_features * 2)
    #concatenated_features = concatenated_features.view(-1, num_features * 2)
    #print(concatenated_features)


    return concatenated_features,center_indices, neighbor_indices

# Example usage
FF = torch.tensor([[1, 2], [3, 4], [5, 6],[8,9],[10,11]], dtype=torch.float32)  # Feature matrix
A = torch.tensor([[0, 1, 0,0,1], [1, 0, 1,0,0], [0, 1, 0,1,1],[0,0,1,0,0],[1,0,1,0,0]], dtype=torch.float32)  # Adjacency matrix

concatenated_features_matrix,center_indices, neighbor_indices = concatenate_features_optimized(FF, A)
#df_concatenated_features = pd.DataFrame(concatenated_features_matrix.numpy())

In [ ]:
model = nn.Sequential(
    nn.Linear(2,2),
    nn.Tanh()
    )

def sort_l1_distance ( model , concatenated_features_matrix,center_indices, neighbor_indices ) :

  concatenated_scores = model(concatenated_features_matrix)
  l1_distances = torch.sum(torch.abs(concatenated_scores[:, 0, :] - concatenated_scores[:, 1, :]), dim=1)
  #
  # Step 2: Sort the L1-distances while keeping track of the indices
  sorted_l1_distances, sorted_indices = torch.sort(l1_distances)
  # Step 3: Use the sorted indices to sort the center and neighbor nodes
  sorted_center_nodes = center_indices[sorted_indices]
  sorted_neighbor_nodes = neighbor_indices[sorted_indices]


  return sorted_center_nodes, sorted_l1_distances, sorted_indices , sorted_neighbor_nodes

sorted_center_nodes, sorted_l1_distances, sorted_indices , sorted_neighbor_nodes = sort_l1_distance (
                                                                                  model ,
                                                                                  concatenated_features_matrix,
                                                                                  center_indices,
                                                                                  neighbor_indices )


In [ ]:
def top_p_sampling(top_ps,adj_mat,sorted_center_nodes,sorted_l1_distances,
                   sorted_indices,center_indices,neighbor_indices):
  sampled_l1_distance_list = []
  sampled_center_node_list = []
  sampled_neigh_node_list = []
  for center_node,center_neighs in enumerate(adj_mat) :
    mask = sorted_center_nodes == center_node
    center_filtered = center_indices[sorted_indices]
    neigh_filtered = neighbor_indices[sorted_indices]
    top_p = top_ps[center_node]
    if top_p * len(neigh_filtered[mask]) > 1 :
      slice_end = int(np.floor(top_p * len(neigh_filtered[mask])))
    else :
      slice_end = None

    sampled_l1_distance = sorted_l1_distances[mask][:slice_end]
    sampled_center_node = center_filtered[mask][:slice_end]
    sampled_neigh_node = neigh_filtered[mask][:slice_end]

    sampled_l1_distance_list.append(sampled_l1_distance)
    sampled_center_node_list.append(sampled_center_node)
    sampled_neigh_node_list.append(sampled_neigh_node)

  return sampled_l1_distance_list, sampled_center_node_list, sampled_neigh_node_list

# sampled_l1_distance, sampled_center_node, sampled_neigh_node = top_p_sampling(top_ps,
#                                                        A,sorted_center_nodes,sorted_l1_distances,
#                                                        sorted_indices,center_indices,neighbor_indices)

In [ ]:
def adj_mat_neighbor_selection (num_nodes , sampled_center_node , sampled_neigh_node ) :
  # Sample parallel lists of indices
  i_indices = torch.cat(sampled_center_node).tolist()
  j_indices = torch.cat(sampled_neigh_node).tolist()

  data = np.ones(len(i_indices))  # All non-zero values are 1

# Assuming the size of the matrix is known or can be derived
  print(num_nodes)
  num_rows = num_nodes
  num_columns = num_nodes

  # Create the sparse matrix
  sparse_matrix = coo_matrix((data, (i_indices, j_indices)), shape=(num_rows, num_columns))

  # Convert the sparse matrix to a dense matrix
  dense_matrix = sparse_matrix.toarray()

  return dense_matrix , sparse_matrix

# dense_matrix , sparse_matrix = adj_mat_neighbor_selection (5 , sampled_center_node , sampled_neigh_node)

In [ ]:
def neighbor_selection (feature_matrix, adjacency_matrix , model) :
  num_nodes = len(adjacency_matrix)
  top_ps =[ 0.5 for i in range(num_nodes)]
  concatenated_features_matrix, center_indices, neighbor_indices = concatenate_features_optimized(feature_matrix, adjacency_matrix)
  sorted_center_nodes, sorted_l1_distances, sorted_indices , sorted_neighbor_nodes = sort_l1_distance (
                                                                                  model ,
                                                                                  concatenated_features_matrix,
                                                                                  center_indices,
                                                                                  neighbor_indices )
  sampled_l1_distance, sampled_center_node, sampled_neigh_node = top_p_sampling(top_ps,
                                                       A,sorted_center_nodes,sorted_l1_distances,
                                                       sorted_indices,center_indices,neighbor_indices)
  dense_matrix , sparse_matrix = adj_mat_neighbor_selection (num_nodes , sampled_center_node , sampled_neigh_node)

  return dense_matrix , sparse_matrix

# dense_matrix , sparse_matrix = neighbor_selection(feature_matrix, adjacency_matrix , model)

In [ ]:
dense_matrix , sparse_matrix = neighbor_selection(FF, A , model)
dense_matrix

5


array([[0., 1., 0., 0., 1.],
       [1., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 1., 0., 0.]])

In [ ]:
def edge_to_dic_adjacency(edge_index):
  dic = {}
  for i in edge_index[0] :
    dic[int(i)] = list()

  for key,value in zip(edge_index[0],edge_index[1]) :
    dic[int(key)].append(int(value))
  return dic

adjacency_list_dict = edge_to_dic_adjacency(edge_index)
adjacency_list_dict_train = edge_to_dic_adjacency(edge_index_train)

NameError: name 'torch' is not defined

In [ ]:
def dic_to_adjmatrix(adjacency_list_dict,num_nodes):
  # adjacency_list_dict
  adj_matrix = np.zeros((num_nodes,num_nodes))
  for node in adjacency_list_dict.keys() :
    for neighbor in adjacency_list_dict[node] :
      adj_matrix[neighbor][node] = 1
  return adj_matrix

adj_matrix = torch.tensor(dic_to_adjmatrix(adjacency_list_dict,11701),dtype = torch.float32)
adj_matrix_train = torch.tensor(dic_to_adjmatrix(adjacency_list_dict_train,11701),dtype = torch.float32)

In [ ]:
model = nn.Sequential(
    nn.Linear(300,2),
    nn.Tanh()
    )
dense_matrix , sparse_matrix = neighbor_selection(data.x, adj_matrix , model)

11701


In [ ]:
# %%time
# test
# a = torch.tensor(dense_matrix) - adj_matrix
# b = a[a<0]
# b
# torch.dot                            # [D], [D] -> []
# batched_dot = torch.vmap(torch.dot)  # [N, D], [N, D] -> [N]
# x, y = torch.randn(100000, 5), torch.randn(100000, 5)
# z = batched_dot(x, y)
# z

In [ ]:
def norm_mat_gen(adj_mat):
  degrees = torch.sum(adj_mat,axis = 1)
  inv_degrees = torch.where(degrees > 0, 1.0 / degrees, torch.zeros_like(degrees))
  norm_mat = torch.diag(inv_degrees)
  return norm_mat

# matrix_test = torch.tensor([[0,0,1,1],[0,0,0,1],[1,0,0,1],[1,1,1,0]],dtype = torch.float32)
norm_mat = norm_mat_gen(adj_matrix)
norm_adj_mat = torch.spmm(norm_mat,adj_matrix)

In [ ]:
class CustomGNNLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(CustomGNNLayer, self).__init__()
        # self.linear = nn.Linear(in_channels, out_channels)
        self.weight = nn.Parameter(torch.FloatTensor(in_features,out_features))
        self.bias = nn.Parameter(torch.FloatTensor(out_features))

        init.xavier_uniform_(self.weight)
        init.zeros_(self.bias)


    def forward(self, x, norm_adj_mat):
        # TODO : add neighbor selection
        x = torch.spmm(norm_adj_mat,x)
        x = torch.mm(x,self.weight)
        x = x + self.bias

        return x

In [ ]:
class NeighborSelectionLayer(nn.Module):
  def __init__ (self):
    super(NeighborSelectionLayer, self).__init__()
    self.model = nn.Sequential(
                nn.Linear(300,2),
                nn.Tanh()
                )
    num_nodes = 11701
    self.top_ps = [ 0.9 for i in range(num_nodes)]
  def forward(self, x, norm_adj_mat):
    new_adj_mat = self.neighbor_selection(x , norm_adj_mat)

    return new_adj_mat

  def concatenate_features_optimized(self ,feature_matrix, adjacency_matrix):
    num_nodes, num_features = feature_matrix.shape

    center_indices, neighbor_indices = torch.nonzero(adjacency_matrix, as_tuple=True)

    center_features = feature_matrix[center_indices]
    neighbor_features = feature_matrix[neighbor_indices]

    concatenated_features = torch.cat((center_features.unsqueeze(1), neighbor_features.unsqueeze(1)), dim=1)

    return concatenated_features,center_indices, neighbor_indices

  def sort_l1_distance (self , concatenated_features_matrix,center_indices, neighbor_indices ) :

    concatenated_scores = self.model(concatenated_features_matrix)
    l1_distances = torch.sum(torch.abs(concatenated_scores[:, 0, :] - concatenated_scores[:, 1, :]), dim=1)

    sorted_l1_distances, sorted_indices = torch.sort(l1_distances)

    sorted_center_nodes = center_indices[sorted_indices]
    sorted_neighbor_nodes = neighbor_indices[sorted_indices]


    return sorted_center_nodes, sorted_l1_distances, sorted_indices , sorted_neighbor_nodes

  def top_p_sampling(self ,adj_mat,sorted_center_nodes,sorted_l1_distances,
                   sorted_indices,center_indices,neighbor_indices):
    sampled_l1_distance_list = []
    sampled_center_node_list = []
    sampled_neigh_node_list = []
    for center_node,center_neighs in enumerate(adj_mat) :
      mask = sorted_center_nodes == center_node
      center_filtered = center_indices[sorted_indices]
      neigh_filtered = neighbor_indices[sorted_indices]
      top_p = self.top_ps[center_node]
      if top_p * len(neigh_filtered[mask]) > 1 :
        slice_end = int(np.floor(top_p * len(neigh_filtered[mask])))
      else :
        slice_end = None

      sampled_l1_distance = sorted_l1_distances[mask][:slice_end]
      sampled_center_node = center_filtered[mask][:slice_end]
      sampled_neigh_node = neigh_filtered[mask][:slice_end]

      sampled_l1_distance_list.append(sampled_l1_distance)
      sampled_center_node_list.append(sampled_center_node)
      sampled_neigh_node_list.append(sampled_neigh_node)

    return sampled_l1_distance_list, sampled_center_node_list, sampled_neigh_node_list

  def adj_mat_neighbor_selection (self ,num_nodes , sampled_center_node , sampled_neigh_node ) :

    i_indices = torch.cat(sampled_center_node).tolist()
    j_indices = torch.cat(sampled_neigh_node).tolist()

    data = np.ones(len(i_indices))

    num_rows = num_nodes
    num_columns = num_nodes

    sparse_matrix = coo_matrix((data, (i_indices, j_indices)), shape=(num_rows, num_columns))

    dense_matrix = sparse_matrix.toarray()

    return dense_matrix , sparse_matrix

  def norm_mat_gen(self , adj_mat):
    degrees = torch.sum(adj_mat,axis = 1)
    inv_degrees = torch.where(degrees > 0, 1.0 / degrees, torch.zeros_like(degrees))
    norm_mat = torch.diag(inv_degrees)
    return norm_mat


  def neighbor_selection (self ,feature_matrix, adjacency_matrix) :
    num_nodes = len(adjacency_matrix)
    concatenated_features_matrix, center_indices, neighbor_indices = self.concatenate_features_optimized(feature_matrix, adjacency_matrix)
    sorted_center_nodes, sorted_l1_distances, sorted_indices , sorted_neighbor_nodes = self.sort_l1_distance (
                                                                                    concatenated_features_matrix,
                                                                                    center_indices,
                                                                                    neighbor_indices )
    sampled_l1_distance, sampled_center_node, sampled_neigh_node = self.top_p_sampling(
                                                        A,sorted_center_nodes,sorted_l1_distances,
                                                        sorted_indices,center_indices,neighbor_indices)
    dense_matrix , sparse_matrix = self.adj_mat_neighbor_selection (num_nodes , sampled_center_node , sampled_neigh_node)

    norm_adj_mat = self.norm_mat_gen(torch.tensor(dense_matrix, dtype = torch.float32))
    return norm_adj_mat


In [ ]:
# TODO : find out if the parameters of MLP layer calculating l1-distance are being updated.


In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = CustomGNNLayer(in_channels, hidden_channels)
        self.conv2 = CustomGNNLayer(hidden_channels, out_channels)
        self.neighb_select = NeighborSelectionLayer()

    def forward(self, x, norm_adj_mat):
        norm_adj_mat = self.neighb_select(x , norm_adj_mat)
        x = self.conv1(x, norm_adj_mat)
        x = F.relu(x)
        x = self.conv2(x, norm_adj_mat)
        return x

In [ ]:
# Initialize the model, optimizer, and loss function
model = GCN(300, 16, 10)
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-12)
criterion = nn.CrossEntropyLoss()
# criterion = nn.MSELoss()
train_negtest = torch.logical_not(data.test_mask)
num_epoch = 150

for epoch in range(num_epoch):
  out = model(data.x, norm_adj_mat)
  # loss = criterion(out, one_hot_labels)
  #train loss
  loss = criterion(out[train_negtest], one_hot_labels[train_negtest])

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  if epoch % 10 == 0:
    print(f'Epoch {epoch}, Loss: {loss:.4f}')

Epoch 0, Loss: 2.3026
Epoch 10, Loss: 2.2472
Epoch 20, Loss: 2.2032
Epoch 30, Loss: 2.1696
Epoch 40, Loss: 2.1446


KeyboardInterrupt: 